In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.12.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate) :
    return tf.keras.layers.Dropout(rate)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=0.5))

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())
        model.add(dropout(rate=0.5))

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\jihyu\anaconda3\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 0.9138, train_loss: 2.33845043, train_accuracy: 0.2891, test_Accuracy: 0.1677
Epoch: [ 0] [    1/  468] time: 1.2653, train_loss: 2.26157284, train_accuracy: 0.2109, test_Accuracy: 0.2239
Epoch: [ 0] [    2/  468] time: 1.6116, train_loss: 2.36649942, train_accuracy: 0.3359, test_Accuracy: 0.2668
Epoch: [ 0] [    3/  468] time: 1.9557, train_loss: 2.34149098, train_accuracy: 0.3125, test_Accuracy: 0.3013
Epoch: [ 0] [    4/  468] time: 2.3184, train_loss: 2.24217033, train_accuracy: 0.3750, test_Accuracy: 0.2899
Epoch: [ 0] [    5/  468] time: 2.6730, train_loss: 2.17957735, train_accuracy: 0.3594, test_Accuracy: 0.3005
Epoch: [ 0] [    6/  468] time: 3.0543, train_loss: 2.18939638, train_accuracy: 0.3516, test_Accuracy: 0.3495
Epoch: [ 0] [    7/  468] time: 3.4549, train_loss: 2.15908456, train_accuracy: 0.4609, test_Accuracy: 0.4063
Epoch: [ 0] [    8/  468] time: 3.8188, train_loss: 2.19803572, train_accuracy: 0.4297, test_Accuracy: 0.4520
Epoch: [ 0

Epoch: [ 0] [   75/  468] time: 29.0154, train_loss: 0.58527154, train_accuracy: 0.8906, test_Accuracy: 0.8843
Epoch: [ 0] [   76/  468] time: 29.4203, train_loss: 0.66479385, train_accuracy: 0.8516, test_Accuracy: 0.8853
Epoch: [ 0] [   77/  468] time: 29.8252, train_loss: 0.62868416, train_accuracy: 0.8828, test_Accuracy: 0.8841
Epoch: [ 0] [   78/  468] time: 30.2197, train_loss: 0.65949571, train_accuracy: 0.8828, test_Accuracy: 0.8851
Epoch: [ 0] [   79/  468] time: 30.6294, train_loss: 0.42709258, train_accuracy: 0.8984, test_Accuracy: 0.8872
Epoch: [ 0] [   80/  468] time: 31.0832, train_loss: 0.56099641, train_accuracy: 0.8828, test_Accuracy: 0.8883
Epoch: [ 0] [   81/  468] time: 31.5096, train_loss: 0.57565475, train_accuracy: 0.8672, test_Accuracy: 0.8915
Epoch: [ 0] [   82/  468] time: 31.8976, train_loss: 0.68435484, train_accuracy: 0.8672, test_Accuracy: 0.8918
Epoch: [ 0] [   83/  468] time: 32.2919, train_loss: 0.66515535, train_accuracy: 0.8984, test_Accuracy: 0.8886
E

Epoch: [ 0] [  149/  468] time: 60.0576, train_loss: 0.48770562, train_accuracy: 0.9219, test_Accuracy: 0.9264
Epoch: [ 0] [  150/  468] time: 60.4800, train_loss: 0.30542344, train_accuracy: 0.9688, test_Accuracy: 0.9251
Epoch: [ 0] [  151/  468] time: 60.8858, train_loss: 0.30894148, train_accuracy: 0.9688, test_Accuracy: 0.9207
Epoch: [ 0] [  152/  468] time: 61.3002, train_loss: 0.42105195, train_accuracy: 0.8906, test_Accuracy: 0.9172
Epoch: [ 0] [  153/  468] time: 61.7157, train_loss: 0.31193608, train_accuracy: 0.9219, test_Accuracy: 0.9145
Epoch: [ 0] [  154/  468] time: 62.1150, train_loss: 0.45513284, train_accuracy: 0.8984, test_Accuracy: 0.9164
Epoch: [ 0] [  155/  468] time: 62.5362, train_loss: 0.32993668, train_accuracy: 0.9141, test_Accuracy: 0.9184
Epoch: [ 0] [  156/  468] time: 62.9765, train_loss: 0.25004959, train_accuracy: 0.9375, test_Accuracy: 0.9217
Epoch: [ 0] [  157/  468] time: 63.4336, train_loss: 0.35978645, train_accuracy: 0.9219, test_Accuracy: 0.9238
E

Epoch: [ 0] [  223/  468] time: 91.5573, train_loss: 0.29420227, train_accuracy: 0.9375, test_Accuracy: 0.9396
Epoch: [ 0] [  224/  468] time: 92.0033, train_loss: 0.30370834, train_accuracy: 0.9531, test_Accuracy: 0.9406
Epoch: [ 0] [  225/  468] time: 92.4382, train_loss: 0.25005212, train_accuracy: 0.9375, test_Accuracy: 0.9399
Epoch: [ 0] [  226/  468] time: 92.8809, train_loss: 0.42075825, train_accuracy: 0.9453, test_Accuracy: 0.9403
Epoch: [ 0] [  227/  468] time: 93.3074, train_loss: 0.25863340, train_accuracy: 0.9531, test_Accuracy: 0.9397
Epoch: [ 0] [  228/  468] time: 93.7332, train_loss: 0.33677304, train_accuracy: 0.9453, test_Accuracy: 0.9399
Epoch: [ 0] [  229/  468] time: 94.1477, train_loss: 0.35052359, train_accuracy: 0.9453, test_Accuracy: 0.9413
Epoch: [ 0] [  230/  468] time: 94.5665, train_loss: 0.33359480, train_accuracy: 0.9297, test_Accuracy: 0.9414
Epoch: [ 0] [  231/  468] time: 95.0119, train_loss: 0.53232867, train_accuracy: 0.8984, test_Accuracy: 0.9421
E

Epoch: [ 0] [  297/  468] time: 123.4790, train_loss: 0.24960217, train_accuracy: 0.9375, test_Accuracy: 0.9436
Epoch: [ 0] [  298/  468] time: 123.9397, train_loss: 0.24360777, train_accuracy: 0.9453, test_Accuracy: 0.9430
Epoch: [ 0] [  299/  468] time: 124.3498, train_loss: 0.26448008, train_accuracy: 0.9688, test_Accuracy: 0.9427
Epoch: [ 0] [  300/  468] time: 124.8053, train_loss: 0.44452366, train_accuracy: 0.9141, test_Accuracy: 0.9436
Epoch: [ 0] [  301/  468] time: 125.2291, train_loss: 0.16025448, train_accuracy: 0.9688, test_Accuracy: 0.9440
Epoch: [ 0] [  302/  468] time: 125.6512, train_loss: 0.41110608, train_accuracy: 0.9297, test_Accuracy: 0.9451
Epoch: [ 0] [  303/  468] time: 126.0696, train_loss: 0.35622329, train_accuracy: 0.9375, test_Accuracy: 0.9453
Epoch: [ 0] [  304/  468] time: 126.5082, train_loss: 0.32138634, train_accuracy: 0.9297, test_Accuracy: 0.9459
Epoch: [ 0] [  305/  468] time: 126.9539, train_loss: 0.30735978, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  371/  468] time: 155.5625, train_loss: 0.42940027, train_accuracy: 0.9219, test_Accuracy: 0.9536
Epoch: [ 0] [  372/  468] time: 156.0312, train_loss: 0.26358044, train_accuracy: 0.9375, test_Accuracy: 0.9542
Epoch: [ 0] [  373/  468] time: 156.4607, train_loss: 0.27000341, train_accuracy: 0.9297, test_Accuracy: 0.9550
Epoch: [ 0] [  374/  468] time: 156.8936, train_loss: 0.24485713, train_accuracy: 0.9219, test_Accuracy: 0.9545
Epoch: [ 0] [  375/  468] time: 157.3434, train_loss: 0.37130028, train_accuracy: 0.9453, test_Accuracy: 0.9543
Epoch: [ 0] [  376/  468] time: 157.7679, train_loss: 0.18111981, train_accuracy: 0.9688, test_Accuracy: 0.9543
Epoch: [ 0] [  377/  468] time: 158.1968, train_loss: 0.29589766, train_accuracy: 0.9531, test_Accuracy: 0.9539
Epoch: [ 0] [  378/  468] time: 158.6216, train_loss: 0.22489570, train_accuracy: 0.9453, test_Accuracy: 0.9537
Epoch: [ 0] [  379/  468] time: 159.1509, train_loss: 0.18144813, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  445/  468] time: 190.8753, train_loss: 0.19757639, train_accuracy: 0.9531, test_Accuracy: 0.9563
Epoch: [ 0] [  446/  468] time: 191.3371, train_loss: 0.21790880, train_accuracy: 0.9375, test_Accuracy: 0.9553
Epoch: [ 0] [  447/  468] time: 191.7732, train_loss: 0.22783631, train_accuracy: 0.9531, test_Accuracy: 0.9546
Epoch: [ 0] [  448/  468] time: 192.1933, train_loss: 0.33523673, train_accuracy: 0.9531, test_Accuracy: 0.9534
Epoch: [ 0] [  449/  468] time: 192.6195, train_loss: 0.18378955, train_accuracy: 0.9609, test_Accuracy: 0.9537
Epoch: [ 0] [  450/  468] time: 193.0650, train_loss: 0.30423513, train_accuracy: 0.9375, test_Accuracy: 0.9543
Epoch: [ 0] [  451/  468] time: 193.4989, train_loss: 0.25867546, train_accuracy: 0.9453, test_Accuracy: 0.9532
Epoch: [ 0] [  452/  468] time: 193.9341, train_loss: 0.28099933, train_accuracy: 0.9453, test_Accuracy: 0.9530
Epoch: [ 0] [  453/  468] time: 194.3773, train_loss: 0.46242937, train_accuracy: 0.9062, test_Accuracy: